In [45]:
%logstop
%logstart -rtq ~/.logs/dw.py append
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

In [3]:
from static_grader import grader

# DW Miniproject
## Introduction

The objective of this miniproject is to exercise your ability to wrangle tabular data set and aggregate large data sets into meaningful summary statistics. We'll work with the same medical data used in the `pw` miniproject but leverage the power of Pandas to more efficiently represent and act on our data.

## Downloading the data

We first need to download the data we'll be using from Amazon S3:

In [4]:
!mkdir dw-data
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/201701scripts_sample.csv.gz -nc -P ./dw-data/
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/201606scripts_sample.csv.gz -nc -P ./dw-data/
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/practices.csv.gz -nc -P ./dw-data/
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/chem.csv.gz -nc -P ./dw-data/

mkdir: cannot create directory ‘dw-data’: File exists
File ‘./dw-data/201701scripts_sample.csv.gz’ already there; not retrieving.

File ‘./dw-data/201606scripts_sample.csv.gz’ already there; not retrieving.

File ‘./dw-data/practices.csv.gz’ already there; not retrieving.

File ‘./dw-data/chem.csv.gz’ already there; not retrieving.



## Loading the data

Similar to the `PW` miniproject, the first step is to read in the data. The data files are stored as compressed CSV files. You can load the data into a Pandas DataFrame by making use of the `gzip` package to decompress the files and Panda's `read_csv` methods to parse the data into a DataFrame. You may want to check the Pandas documentation for parsing [CSV](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) files for reference.

For a description of the data set please, refer to the [PW miniproject](./pw.ipynb). **Note that all questions make use of the 2017 data only, except for Question 5 which makes use of both the 2017 and 2016 data.**

In [5]:
import pandas as pd
import numpy as np
import gzip
import json

In [6]:
# load the 2017 data
with gzip.open("dw-data/201701scripts_sample.csv.gz", 'rb') as f:
    scripts = pd.read_csv(f)
scripts.shape

(973193, 7)

In [7]:
col_names=[ 'code', 'name', 'addr_1', 'addr_2', 'borough', 'village', 'post_code']
with gzip.open('dw-data/practices.csv.gz', 'rb') as f:
    practices = pd.read_csv(f,header=None,names=col_names)
practices.shape

(12020, 7)

In [8]:
with gzip.open("dw-data/chem.csv.gz","rb") as f:
    chem=pd.read_csv(f)
chem.shape

(3487, 2)

Now that we've loaded in the data, let's first replicate our results from the `PW` miniproject. Note that we are now working with a larger data set so the answers will be different than in the `PW` miniproject even if the analysis is the same.

## Question 1: summary_statistics

In the `PW` miniproject we first calculated the total, mean, standard deviation, and quartile statistics of the `'items'`, `'quantity'`', `'nic'`, and `'act_cost'` fields. To do this we had to write some functions to calculate the statistics and apply the functions to our data structure. The DataFrame has a `describe` method that will calculate most (not all) of these things for us.

Submit the summary statistics to the grader as a list of tuples: [('act_cost', (total, mean, std, q25, median, q75)), ...]

In [10]:
scripts.head()

,practice,bnf_code,bnf_name,items,nic,act_cost,quantity
0,N85639,0106020C0,Bisacodyl_Tab E/C 5mg,1,0.39,0.47,12
1,N85639,0106040M0,Movicol Plain_Paed Pdr Sach 6.9g,1,4.38,4.07,30
2,N85639,0301011R0,Salbutamol_Inha 100mcg (200 D) CFF,1,1.50,1.40,1
3,N85639,0304010G0,Chlorphenamine Mal_Oral Soln 2mg/5ml,1,2.62,2.44,150
4,N85639,0401020K0,Diazepam_Tab 2mg,1,0.16,0.26,6


In [11]:
s=scripts.describe()
s.loc["total"]={"items":scripts["items"].sum(),"nic":scripts["nic"].sum(),"act_cost":scripts["act_cost"].sum(),"quantity":scripts["quantity"].sum()}


In [12]:
s=s.loc[["total","mean","std","25%","50%","75%"]]

In [13]:
s=s[["items","quantity","nic",'act_cost']]

In [14]:
tuple(s["items"])

(8888304.0, 9.133135976111625, 29.204198282803603, 1.0, 2.0, 6.0)

In [15]:
def stat(df):
    lst=[]
    for t in df.columns:
        lst.append((t,tuple(df[t])))
    return lst
print(stat(s))
    
    
    

[('items', (8888304.0, 9.133135976111625, 29.204198282803603, 1.0, 2.0, 6.0)), ('quantity', (721457006.0, 741.3298348837282, 3665.426958467915, 28.0, 100.0, 350.0)), ('nic', (71100424.84000002, 73.05891517920908, 188.070256906825, 7.8, 22.64, 65.0)), ('act_cost', (66164096.11999999, 67.98661326170655, 174.40170332301963, 7.33, 21.22, 60.67))]


In [16]:
s.index

Index(['total', 'mean', 'std', '25%', '50%', '75%'], dtype='object')

In [17]:
tuple(s)

('items', 'quantity', 'nic', 'act_cost')

In [18]:
summary_stats = stat(s)

In [19]:
summary_stats

[('items', (8888304.0, 9.133135976111625, 29.204198282803603, 1.0, 2.0, 6.0)),
 ('quantity',
  (721457006.0, 741.3298348837282, 3665.426958467915, 28.0, 100.0, 350.0)),
 ('nic',
  (71100424.84000002, 73.05891517920908, 188.070256906825, 7.8, 22.64, 65.0)),
 ('act_cost',
  (66164096.11999999,
   67.98661326170655,
   174.40170332301963,
   7.33,
   21.22,
   60.67))]

In [20]:
grader.score.dw__summary_statistics(summary_stats)

Your score:  1.0


## Question 2: most_common_item

We can also easily compute summary statistics on groups within the data. In the `pw` miniproject we had to explicitly construct the groups based on the values of a particular field. Pandas will handle that for us via the `groupby` method. This process is [detailed in the Pandas documentation](https://pandas.pydata.org/pandas-docs/stable/groupby.html).

Use `groupby` to calculate the total number of items dispensed for each `'bnf_name'`. Find the item with the highest total and return the result as `[(bnf_name, total)]`.

In [21]:
scripts.head()

,practice,bnf_code,bnf_name,items,nic,act_cost,quantity
0,N85639,0106020C0,Bisacodyl_Tab E/C 5mg,1,0.39,0.47,12
1,N85639,0106040M0,Movicol Plain_Paed Pdr Sach 6.9g,1,4.38,4.07,30
2,N85639,0301011R0,Salbutamol_Inha 100mcg (200 D) CFF,1,1.50,1.40,1
3,N85639,0304010G0,Chlorphenamine Mal_Oral Soln 2mg/5ml,1,2.62,2.44,150
4,N85639,0401020K0,Diazepam_Tab 2mg,1,0.16,0.26,6


In [22]:
scripts["bnf_name"].nunique()

13471

In [23]:
dir(scripts.groupby("bnf_name"))

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_accessors',
 '_add_numeric_operations',
 '_agg_examples_doc',
 '_agg_see_also_doc',
 '_aggregate',
 '_aggregate_generic',
 '_aggregate_item_by_item',
 '_aggregate_multiple_funcs',
 '_apply_filter',
 '_apply_to_column_groupbys',
 '_apply_whitelist',
 '_assure_grouper',
 '_block_agg_axis',
 '_bool_agg',
 '_builtin_table',
 '_choose_path',
 '_concat_objects',
 '_constructor',
 '_cumcount_array',
 '_cython_agg_blocks',
 '_cython_agg_general',
 '_cython_table',
 '_cython_transform',
 '_decide_output_index',
 '_def_str',
 '_define_paths',
 '_deprecations',
 '_dir_additi

In [24]:
new_s=scripts.groupby("bnf_name").sum()["items"]

In [25]:
new_s.head()

bnf_name
365 Film 10cm x 12cm VP Adh Film Dress      2
365 Non Adherent 10cm x 10cm Pfa Plas Fa    3
365 Non Adherent 10cm x 20cm Pfa Plas Fa    1
365 Non Woven Island 8cm x 10cm Adh Dres    1
365 Transpt Island 5cm x 7.2cm VP Adh Fi    2
Name: items, dtype: int64

In [26]:
[(new_s.idxmax(),new_s.max())]

[('Omeprazole_Cap E/C 20mg', 218583)]

In [27]:
most_common_item = [(new_s.idxmax(),new_s.max())]

In [28]:
grader.score.dw__most_common_item(most_common_item)

Your score:  1.0


## Question 3: items_by_region

Now let's find the most common item by post code. The post code information is in the `practices` DataFrame, and we'll need to `merge` it into the `scripts` DataFrame. Pandas provides [extensive documentation](https://pandas.pydata.org/pandas-docs/stable/merging.html) with diagrammed examples on different methods and approaches for joining data. The `merge` method is only one of many possible options.

Return your results as a list of tuples `(post code, item name, amount dispensed as % of total)`. Sort your results ascending alphabetically by post code and take only results from the first 100 post codes.

**NOTE:** Some practices have multiple postal codes associated with them. Use the alphabetically first postal code. Note some postal codes may have multiple `'bnf_name'` with the same prescription rate for the maximum. In this case, take the alphabetically first `'bnf_name'` (as in the PW miniproject).

In [29]:
practices.head()

,code,name,addr_1,addr_2,borough,village,post_code
0,A81001,THE DENSHAM SURGERY,THE HEALTH CENTRE,LAWSON STREET,STOCKTON ON TEES,CLEVELAND,TS18 1HU
1,A81002,QUEENS PARK MEDICAL CENTRE,QUEENS PARK MEDICAL CTR,FARRER STREET,STOCKTON ON TEES,CLEVELAND,TS18 2AW
2,A81003,VICTORIA MEDICAL PRACTICE,THE HEALTH CENTRE,VICTORIA ROAD,HARTLEPOOL,CLEVELAND,TS26 8DB
3,A81004,WOODLANDS ROAD SURGERY,6 WOODLANDS ROAD,NaN,MIDDLESBROUGH,CLEVELAND,TS1 3BE
4,A81005,SPRINGWOOD SURGERY,SPRINGWOOD SURGERY,RECTORY LANE,GUISBOROUGH,NaN,TS14 7DJ


In [30]:
#taking alphabetically first postal code

df1=practices[["code","post_code"]]
df1=df1.set_index("code").sort_values(by="post_code").sort_index().reset_index()
df1.head()

,code,post_code
0,A81001,TS18 1HU
1,A81001,TS18 1HU
2,A81002,TS18 2AW
3,A81003,TS25 1QU
4,A81003,TS26 8DB


In [31]:
df1=df1.drop_duplicates("code",keep="first")

df1.head()


,code,post_code
0,A81001,TS18 1HU
2,A81002,TS18 2AW
3,A81003,TS25 1QU
5,A81004,TS1 3BE
7,A81005,TS14 7DJ


In [32]:
df1.shape

(10843, 2)

In [33]:

df2=scripts.set_index("practice").sort_index().reset_index()
df2.head()

,practice,bnf_code,bnf_name,items,nic,act_cost,quantity
0,A81005,0409010H0,Ropinirole HCl_Tab 8mg M/R,4,336.88,311.91,224
1,A81005,0407010F0,Co-Codamol Eff_Tab 30mg/500mg,1,17.32,16.05,200
2,A81005,0407010F0,Co-Codamol_Tab 30mg/500mg,33,153.53,143.16,3866
3,A81005,0407010F0,Co-Codamol_Tab 15mg/500mg,8,73.95,69.05,772
4,A81005,0407010F0,Co-Codamol_Cap 15mg/500mg,3,30.74,28.59,424


In [34]:
#merging the the "script" and 'practice' dataframes

merged_df=df2.merge(df1,how="left",left_on="practice",right_on="code")
merged_df.head()

,practice,bnf_code,bnf_name,items,nic,act_cost,quantity,code,post_code
0,A81005,0409010H0,Ropinirole HCl_Tab 8mg M/R,4,336.88,311.91,224,A81005,TS14 7DJ
1,A81005,0407010F0,Co-Codamol Eff_Tab 30mg/500mg,1,17.32,16.05,200,A81005,TS14 7DJ
2,A81005,0407010F0,Co-Codamol_Tab 30mg/500mg,33,153.53,143.16,3866,A81005,TS14 7DJ
3,A81005,0407010F0,Co-Codamol_Tab 15mg/500mg,8,73.95,69.05,772,A81005,TS14 7DJ
4,A81005,0407010F0,Co-Codamol_Cap 15mg/500mg,3,30.74,28.59,424,A81005,TS14 7DJ


In [35]:

merged_df=merged_df[["practice", "bnf_code", "bnf_name", "items", "post_code"]]
merged_df.head()

,practice,bnf_code,bnf_name,items,post_code
0,A81005,0409010H0,Ropinirole HCl_Tab 8mg M/R,4,TS14 7DJ
1,A81005,0407010F0,Co-Codamol Eff_Tab 30mg/500mg,1,TS14 7DJ
2,A81005,0407010F0,Co-Codamol_Tab 30mg/500mg,33,TS14 7DJ
3,A81005,0407010F0,Co-Codamol_Tab 15mg/500mg,8,TS14 7DJ
4,A81005,0407010F0,Co-Codamol_Cap 15mg/500mg,3,TS14 7DJ


In [36]:
#calculating the total items in each post

total_items_by_post=merged_df.groupby("post_code").agg({"items":"sum"})
total_items_by_post.head()

,items
post_code,
B11 4BW,22731
B12 9LP,17073
B18 7AL,20508
B21 9RY,31027
B23 6DJ,28005


In [37]:
#finding the most common item in each post

merged_df=merged_df.groupby(["post_code","bnf_name"]).agg({"items":"sum"}).reset_index().sort_values("items",ascending=False).drop_duplicates("post_code").set_index("post_code").sort_index()
print(merged_df.shape)
merged_df.head()

(276, 2)


,bnf_name,items
post_code,,
B11 4BW,Salbutamol_Inha 100mcg (200 D) CFF,706
B12 9LP,Paracet_Tab 500mg,425
B18 7AL,Salbutamol_Inha 100mcg (200 D) CFF,556
B21 9RY,Metformin HCl_Tab 500mg,1033
B23 6DJ,Lansoprazole_Cap 30mg (E/C Gran),599


In [38]:
#making a new column with "total" value

merged_df["total_items"]=total_items_by_post["items"]
merged_df.head()

,bnf_name,items,total_items
post_code,,,
B11 4BW,Salbutamol_Inha 100mcg (200 D) CFF,706,22731
B12 9LP,Paracet_Tab 500mg,425,17073
B18 7AL,Salbutamol_Inha 100mcg (200 D) CFF,556,20508
B21 9RY,Metformin HCl_Tab 500mg,1033,31027
B23 6DJ,Lansoprazole_Cap 30mg (E/C Gran),599,28005


In [39]:
#making a new column with the percentage ratio

merged_df["percentage_of_total"]=merged_df["items"]/merged_df["total_items"]
merged_df.head()

,bnf_name,items,total_items,percentage_of_total
post_code,,,,
B11 4BW,Salbutamol_Inha 100mcg (200 D) CFF,706,22731,0.031059
B12 9LP,Paracet_Tab 500mg,425,17073,0.024893
B18 7AL,Salbutamol_Inha 100mcg (200 D) CFF,556,20508,0.027111
B21 9RY,Metformin HCl_Tab 500mg,1033,31027,0.033294
B23 6DJ,Lansoprazole_Cap 30mg (E/C Gran),599,28005,0.021389


In [40]:

result=merged_df[["bnf_name","percentage_of_total"]]

In [41]:
list(result.itertuples(name=None))[:3]

[('B11 4BW', 'Salbutamol_Inha 100mcg (200 D) CFF', 0.031058906339360346),
 ('B12 9LP', 'Paracet_Tab 500mg', 0.02489310607391788),
 ('B18 7AL', 'Salbutamol_Inha 100mcg (200 D) CFF', 0.027111371172225472)]

In [42]:
#making a list of tuples and taking the first 100 value alphabetically

items_by_region = list(result.itertuples(name=None))[:100]

In [43]:
grader.score.dw__items_by_region(items_by_region)

Your score:  0.9600000000000006


## Question 4: script_anomalies

Drug abuse is a source of human and monetary costs in health care. A first step in identifying practitioners that enable drug abuse is to look for practices where commonly abused drugs are prescribed unusually often. Let's try to find practices that prescribe an unusually high amount of opioids. The opioids we'll look for are given in the list below.

In [10]:
opioids = ['morphine', 'oxycodone', 'methadone', 'fentanyl', 'pethidine', 'buprenorphine', 'propoxyphene', 'codeine']

These are generic names for drugs, not brand names. Generic drug names can be found using the `'bnf_code'` field in `scripts` along with the `chem` table.. Use the list of opioids provided above along with these fields to make a new field in the `scripts` data that flags whether the row corresponds with a opioid prescription.

In [11]:
chem.head()

,CHEM SUB,NAME
0,0101010A0,Alexitol Sodium
1,0101010B0,Almasilate
2,0101010C0,Aluminium Hydroxide
3,0101010D0,Aluminium Hydroxide With Magnesium
4,0101010E0,Hydrotalcite


In [12]:
scripts.head()

,practice,bnf_code,bnf_name,items,nic,act_cost,quantity
0,N85639,0106020C0,Bisacodyl_Tab E/C 5mg,1,0.39,0.47,12
1,N85639,0106040M0,Movicol Plain_Paed Pdr Sach 6.9g,1,4.38,4.07,30
2,N85639,0301011R0,Salbutamol_Inha 100mcg (200 D) CFF,1,1.50,1.40,1
3,N85639,0304010G0,Chlorphenamine Mal_Oral Soln 2mg/5ml,1,2.62,2.44,150
4,N85639,0401020K0,Diazepam_Tab 2mg,1,0.16,0.26,6


In [13]:
ops = '|'.join(opioids)

bnf_by_practice = scripts.groupby('practice').agg({'bnf_code': 'count'}).rename(columns={'bnf_code':'bnf_sum'}) #counting the total bnf_codes in each practice
bnf_by_practice.head()

,bnf_sum
practice,
A81005,1507
A81007,1454
A81011,1568
A81012,1332
A81017,2150


In [14]:
m_df = pd.merge(scripts, chem, left_on='bnf_code', right_on='CHEM SUB', how='left', sort=False) #merging two dataframe 
m_df.head()

,practice,bnf_code,bnf_name,items,nic,act_cost,quantity,CHEM SUB,NAME
0,N85639,0106020C0,Bisacodyl_Tab E/C 5mg,1,0.39,0.47,12,0106020C0,Bisacodyl
1,N85639,0106040M0,Movicol Plain_Paed Pdr Sach 6.9g,1,4.38,4.07,30,0106040M0,Macrogol 3350
2,N85639,0301011R0,Salbutamol_Inha 100mcg (200 D) CFF,1,1.50,1.40,1,0301011R0,Salbutamol
3,N85639,0304010G0,Chlorphenamine Mal_Oral Soln 2mg/5ml,1,2.62,2.44,150,0304010G0,Chlorphenamine Maleate
4,N85639,0401020K0,Diazepam_Tab 2mg,1,0.16,0.26,6,0401020K0,Diazepam


In [15]:
m_df['flag'] = (m_df['NAME'].str.contains(ops, case=False) | m_df['bnf_name'].str.contains(ops, case=False))  #checking if any practice have the "opioids"
m_df.head()

,practice,bnf_code,bnf_name,items,nic,act_cost,quantity,CHEM SUB,NAME,flag
0,N85639,0106020C0,Bisacodyl_Tab E/C 5mg,1,0.39,0.47,12,0106020C0,Bisacodyl,False
1,N85639,0106040M0,Movicol Plain_Paed Pdr Sach 6.9g,1,4.38,4.07,30,0106040M0,Macrogol 3350,False
2,N85639,0301011R0,Salbutamol_Inha 100mcg (200 D) CFF,1,1.50,1.40,1,0301011R0,Salbutamol,False
3,N85639,0304010G0,Chlorphenamine Mal_Oral Soln 2mg/5ml,1,2.62,2.44,150,0304010G0,Chlorphenamine Maleate,False
4,N85639,0401020K0,Diazepam_Tab 2mg,1,0.16,0.26,6,0401020K0,Diazepam,False


Now for each practice calculate the proportion of its prescriptions containing opioids.

**Hint:** Consider the following list: `[0, 1, 1, 0, 0, 0]`. What proportion of the entries are 1s? What is the mean value?

In [16]:
opioids_per_practice = m_df.groupby("practice")["flag"].mean()
total_opioid_rate=m_df["flag"].mean()



How do these proportions compare to the overall opioid prescription rate? Subtract off the proportion of all prescriptions that are opioids from each practice's proportion.

In [19]:
relative_opioids_per_practice = opioids_per_practice-total_opioid_rate
total_opioids_std = m_df['flag'].std()
total_opioids_std

0.1857487886805953

Now that we know the difference between each practice's opioid prescription rate and the overall rate, we can identify which practices prescribe opioids at above average or below average rates. However, are the differences from the overall rate important or just random deviations? In other words, are the differences from the overall rate big or small?

To answer this question we have to quantify the difference we would typically expect between a given practice's opioid prescription rate and the overall rate. This quantity is called the **standard error**, and is related to the **standard deviation**, $\sigma$. The standard error in this case is

$$ \frac{\sigma}{\sqrt{n}} $$

where $n$ is the number of prescriptions each practice made. Calculate the standard error for each practice. Then divide `relative_opioids_per_practice` by the standard errors. We'll call the final result `opioid_scores`.

In [20]:
standard_error_per_practice =total_opioids_std / np.sqrt(bnf_by_practice.bnf_sum)
opioid_scores = (relative_opioids_per_practice / standard_error_per_practice).reset_index(name='opioid_scores')
opioid_scores.head()

,practice,opioid_scores
0,A81005,-0.553511
1,A81007,1.536807
2,A81011,2.290292
3,A81012,1.371001
4,A81017,0.583831


The quantity we have calculated in `opioid_scores` is called a **z-score**:

$$ \frac{\bar{X} - \mu}{\sqrt{\sigma^2/n}} $$

Here $\bar{X}$ corresponds with the proportion for each practice, $\mu$ corresponds with the proportion across all practices, $\sigma^2$ corresponds with the variance of the proportion across all practices, and $n$ is the number of prescriptions made by each practice. Notice $\bar{X}$ and $n$ will be different for each practice, while $\mu$ and $\sigma$ are determined across all prescriptions, and so are the same for every z-score. The z-score is a useful statistical tool used for hypothesis testing, finding outliers, and comparing data about different types of objects or events.

Now that we've calculated this statistic, take the 100 practices with the largest z-score. Return your result as a list of tuples in the form `(practice_name, z-score, number_of_scripts)`. Sort your tuples by z-score in descending order. Note that some practice codes will correspond with multiple names. In this case, use the first match when sorting names alphabetically.

In [21]:
# unique_practices = ...
# anomalies = [("NATIONAL ENHANCED SERVICE", 11.6958178629, 7)] * 100
unique_practices = practices.sort_values(by='name').groupby('code', sort=False).first()
unique_practices.reset_index(inplace=True)

merged_df = opioid_scores.merge(unique_practices[['code','name']], left_on='practice',
                                  right_on='code', how='left')

results = merged_df.merge(bnf_by_practice.reset_index(), on='practice')

subset = results[['name', 'opioid_scores', 'bnf_sum']]
anomalies = [tuple(val) for val in subset.values]
anomalies = sorted(anomalies, key=lambda x: x[1], reverse=True)[:100]

In [22]:
results.head()

,practice,opioid_scores,code,name,bnf_sum
0,A81005,-0.553511,A81005,SPRINGWOOD SURGERY,1507
1,A81007,1.536807,A81007,BANKHOUSE SURGERY,1454
2,A81011,2.290292,A81011,CHADWICK PRACTICE,1568
3,A81012,1.371001,A81012,WESTBOURNE MEDICAL CENTRE,1332
4,A81017,0.583831,A81017,WOODBRIDGE PRACTICE,2150


In [23]:
grader.score.dw__script_anomalies(anomalies)

Your score:  0.9900000000000007


## Question 5: script_growth

Another way to identify anomalies is by comparing current data to historical data. In the case of identifying sites of drug abuse, we might compare a practice's current rate of opioid prescription to their rate 5 or 10 years ago. Unless the nature of the practice has changed, the profile of drugs they prescribe should be relatively stable. We might also want to identify trends through time for business reasons, identifying drugs that are gaining market share. That's what we'll do in this question.

We'll load in beneficiary data from 6 months earlier, June 2016, and calculate the percent growth in prescription rate from June 2016 to January 2017 for each `bnf_name`. We'll return the 50 items with largest growth and the 50 items with the largest shrinkage (i.e. negative percent growth) as a list of tuples sorted by growth rate in descending order in the format `(script_name, growth_rate, raw_2016_count)`. You'll notice that many of the 50 fastest growing items have low counts of prescriptions in 2016. Filter out any items that were prescribed less than 50 times.

In [24]:
scripts16 = pd.read_csv('./dw-data/201606scripts_sample.csv.gz')
scripts16.head()

,practice,bnf_code,bnf_name,items,nic,act_cost,quantity
0,N85638,0301011R0,Salamol_Inha 100mcg (200 D) CFF (Teva),2,2.92,2.73,2
1,N85638,0301011R0,Easyhaler_Salbutamol Sulf 200mcg (200D),1,6.63,6.15,1
2,N85638,0301020I0,Ipratrop Brom_Inh Soln 500mcg/2ml Ud,1,1.77,1.75,12
3,N85638,0301020I0,Ipratrop Brom_Inh Soln 250mcg/1ml Ud,1,4.47,4.15,20
4,N85638,0302000C0,Clenil Modulite_Inha 50mcg (200D),1,3.70,3.44,1


In [25]:
scripts.bnf_name.value_counts()

GlucoRX FinePoint Needles Pen Inj Screw     1718
3m Health Care_Cavilon Durable Barrier C     816
Prednisolone_Tab 5mg                         785
Fluclox Sod_Cap 500mg                        783
Amoxicillin_Cap 500mg                        777
                                            ... 
Pot Chlor_Liq Spec 375mg/5ml                   1
ActiLymph Hose Kit Ex Exlge Stkn + Compr       1
Lloyds_A-Z Multivit + Mineral Tab              1
Empagliflozin/Metformin_Tab 12.5mg/850mg       1
Avoca_Silver Nitrate Applic 95%                1
Name: bnf_name, Length: 13471, dtype: int64

In [29]:
# calculating the growth rate
growth = (scripts.bnf_name.value_counts() - scripts16.bnf_name.value_counts())/scripts16.bnf_name.value_counts()
growth.head()

365 Film 10cm x 12cm VP Adh Film Dress      NaN
365 Film 15cm x 20cm VP Adh Film Dress      NaN
365 Film 4cm x 5cm VP Adh Film Dress        NaN
365 Non Adherent 10cm x 10cm Pfa Plas Fa    0.0
365 Non Adherent 10cm x 20cm Pfa Plas Fa    NaN
Name: bnf_name, dtype: float64

In [30]:
# Combining the two series in a Dataframe
resp = pd.DataFrame(dict(growth = growth, item16 = scripts16.bnf_name.value_counts())).reset_index()
resp.head()

,index,growth,item16
0,365 Film 10cm x 12cm VP Adh Film Dress,NaN,NaN
1,365 Film 15cm x 20cm VP Adh Film Dress,NaN,1.0
2,365 Film 4cm x 5cm VP Adh Film Dress,NaN,1.0
3,365 Non Adherent 10cm x 10cm Pfa Plas Fa,0.0,3.0
4,365 Non Adherent 10cm x 20cm Pfa Plas Fa,NaN,NaN


In [31]:
#Removing elements with less then 50 prescriptions
resp = resp[resp['item16'] >= 50]

In [32]:
#Getting rid of Nans
resp.fillna(0, inplace=True)


In [33]:
#Sorting by growth rate
resp.sort_values(by='growth', ascending = False, inplace=True)

In [34]:
#Picking the top 50 and last 50 and get the tuple list
script_growth = list(pd.concat([resp.head(50), resp.tail(50)]).itertuples(index=False, name=None))

In [ ]:
# script_growth = [("Butec_Transdermal Patch 5mcg\/hr", 3.4677419355, 62.0)] * 100

In [35]:
grader.score.dw__script_growth(script_growth)

Your score:  1.0


## Question 6: rare_scripts

Does a practice's prescription costs originate from routine care or from reliance on rarely prescribed treatments? Commonplace treatments can carry lower costs than rare treatments because of efficiencies in large-scale production. While some specialist practices can't help but avoid prescribing rare medicines because there are no alternatives, some practices may be prescribing a unnecessary amount of brand-name products when generics are available. Let's identify practices whose costs disproportionately originate from rarely prescribed items.

First we have to identify which `'bnf_code'` are rare. To do this, find the probability $p$ of a prescription having a particular `'bnf_code'` if the `'bnf_code'` was randomly chosen from the unique options in the beneficiary data. We will call a `'bnf_code'` rare if it is prescribed at a rate less than $0.1p$.

In [36]:
p = 1/scripts.bnf_code.nunique()

rates = scripts['bnf_code'].value_counts() / len(scripts)

mask = rates < .1 * p

rare_codes = rates[mask].index.unique()

scripts['rare'] = scripts['bnf_code'].isin(rare_codes)
scripts.head()

,practice,bnf_code,bnf_name,items,nic,act_cost,quantity,rare
0,N85639,0106020C0,Bisacodyl_Tab E/C 5mg,1,0.39,0.47,12,False
1,N85639,0106040M0,Movicol Plain_Paed Pdr Sach 6.9g,1,4.38,4.07,30,False
2,N85639,0301011R0,Salbutamol_Inha 100mcg (200 D) CFF,1,1.50,1.40,1,False
3,N85639,0304010G0,Chlorphenamine Mal_Oral Soln 2mg/5ml,1,2.62,2.44,150,False
4,N85639,0401020K0,Diazepam_Tab 2mg,1,0.16,0.26,6,False


Now for each practice, calculate the proportion of costs that originate from prescription of rare treatments (i.e. rare `'bnf_code'`). Use the `'act_cost'` field for this calculation.

In [37]:
# rare script act_cost per practice/total act_cost per practice
rare_cost_prop = (scripts[scripts['rare']].groupby('practice')['act_cost'].sum()/scripts.groupby('practice')['act_cost'].sum()).fillna(0)
rare_cost_prop.head()

practice
A81005    0.012017
A81007    0.008381
A81011    0.005116
A81012    0.013747
A81017    0.007359
Name: act_cost, dtype: float64

Now we will calculate a z-score for each practice based on this proportion.
First take the difference of `rare_cost_prop` and the proportion of costs originating from rare treatments across all practices.

In [38]:
relative_rare_cost_prop = (rare_cost_prop - scripts[scripts['rare']]['act_cost'].sum() /scripts['act_cost'].sum())
relative_rare_cost_prop.head()

practice
A81005   -0.003946
A81007   -0.007582
A81011   -0.010847
A81012   -0.002216
A81017   -0.008604
Name: act_cost, dtype: float64

Now we will estimate the standard errors (i.e. the denominator of the z-score) by simply taking the standard deviation of this difference.

In [39]:
standard_errors = relative_rare_cost_prop.std()
standard_errors

0.06050888706745139

Finally compute the z-scores. Return the practices with the top 100 z-scores in the form `(post_code, practice_name, z-score)`. Note that some practice codes will correspond with multiple names. In this case, use the first match when sorting names alphabetically.

In [40]:
rare_scores = relative_rare_cost_prop/standard_errors
rare_scores.head()

practice
A81005   -0.065216
A81007   -0.125308
A81011   -0.179263
A81012   -0.036615
A81017   -0.142190
Name: act_cost, dtype: float64

In [41]:
unique_practices = (practices.sort_values('name').groupby('code').first().reset_index())[['code', 'name']].rename(columns={'code':'practice'})

unique_practices.set_index('practice', inplace=True)

rare_scores = pd.concat([unique_practices, rare_scores], axis =1).sort_values(by='act_cost', ascending=False).head(100)
rare_scores.head()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """


,name,act_cost
Y03472,CONSULTANT DIABETES TEAM,16.262687
Y05320,DMC COMMUNITY DERMATOLOGY RBWF,15.128648
Y04404,OUTPATIENTS JUBILEE HEALTH CENTRE,7.542139
Y03484,DMC COMMUNITY DERMATOLOGY CLINIC,7.287222
Y04424,DMC HEALTHCARE,6.838614


In [ ]:
# rare_scripts = [("Y03472", "CONSULTANT DIABETES TEAM", 16.2626871247)] * 100

In [42]:
rare_scripts = list(rare_scores.itertuples(name=None))

In [43]:
grader.score.dw__rare_scripts(rare_scripts)

Your score:  1.0


*Copyright &copy; 2020 The Data Incubator.  All rights reserved.*